## Generación de Distancias y Rutas entre Ciudades de Colombia
Este cuaderno utiliza la API de OpenRouteService para calcular distancias en carretera entre ciudades colombianas. Genera un archivo CSV con las distancias entre pares de ciudades y un mapa interactivo en HTML que visualiza las rutas.

In [1]:
%pip install openrouteservice folium geopandas


In [14]:
"""
Este script utiliza la API de OpenRouteService para calcular las distancias en carretera entre ciudades colombianas.
Genera un archivo CSV con las distancias entre pares de ciudades y un mapa interactivo en HTML que visualiza las rutas.

Funciones y elementos principales:

1. Autenticación con OpenRouteService:
    - Se utiliza una clave de API (`api_key`) para autenticarse y acceder a los datos de rutas.
    - `client` es una instancia de `openrouteservice.Client`, la cual maneja las solicitudes a la API.

2. Definición de ciudades y coordenadas:
    - El diccionario `ciudades` contiene las coordenadas geográficas de cada ciudad en Colombia que se incluirá en el cálculo de rutas.
    - Las coordenadas están en formato de tuplas (latitud, longitud).

3. Creación del mapa:
    - `m` es una instancia de `folium.Map`, centrada en Bogotá, Colombia, y con un nivel de zoom de inicio de 6.
    - A medida que se recorre el diccionario de `ciudades`, se agregan marcadores (`folium.Marker`) al mapa, cada uno ubicado en la posición geográfica de una ciudad.

4. Escritura de archivo CSV:
    - Se abre el archivo `distancias_ciudades_colombia.csv` en modo escritura (`'w'`) para registrar las distancias calculadas.
    - Un encabezado (`['Origen', 'Destino', 'Distancia (km)']`) es escrito en el archivo para indicar las columnas de datos: la ciudad de origen, la ciudad de destino y la distancia en kilómetros.

5. Cálculo de rutas y distancias:
    - Se recorren todas las combinaciones posibles de ciudades usando bucles anidados.
    - Para cada combinación de `origen` y `destino`:
        - Si `origen` y `destino` son diferentes, el programa solicita una ruta entre las ciudades usando la función `client.directions`.
        - La distancia de la ruta, en metros, se obtiene de la respuesta JSON (`ruta['features'][0]['properties']['segments'][0]['distance']`) y se convierte a kilómetros dividiendo por 1000.
        - La distancia calculada se redondea a dos decimales y se escribe en el archivo CSV.
        - Además, se agrega una capa `GeoJson` al mapa para visualizar la ruta entre cada par de ciudades en `m`.

6. Manejo de errores:
    - Si la API no encuentra una ruta entre dos ciudades (error `Route could not be found`), se escribe 'NA' en el CSV en lugar de una distancia.
    - Cualquier otro error desconocido de la API es impreso en la consola.

7. Guardado del mapa y archivo CSV:
    - Una vez calculadas todas las rutas y distancias, el mapa se guarda como un archivo HTML (`mapa_rutas_colombia.html`), el cual puede abrirse en un navegador para ver las rutas visualmente.
    - Mensajes en la consola indican que el archivo del mapa HTML y el archivo CSV con distancias fueron generados exitosamente.

Dependencias:
    - `openrouteservice`: Permite la conexión con la API de OpenRouteService para obtener rutas y distancias en carretera.
    - `folium`: Crea mapas interactivos en HTML.
    - `csv`: Módulo estándar para la escritura de archivos CSV.

Variables:
    - `api_key` (str): Clave de API para OpenRouteService.
    - `client` (openrouteservice.Client): Cliente de OpenRouteService utilizado para realizar solicitudes de rutas.
    - `ciudades` (dict): Diccionario que contiene nombres de ciudades colombianas como claves y sus coordenadas geográficas como valores.
    - `m` (folium.Map): Mapa interactivo centrado en Bogotá.
    - `archivo_csv` (file): Archivo CSV donde se almacenan las distancias entre las ciudades de Colombia.
"""


import openrouteservice
import folium
import csv

api_key = '5b3ce3597851110001cf624898fde9c1142f454ca9c18c4e0ae0ba2c'
client = openrouteservice.Client(key=api_key)

ciudades = {
    'Leticia': (-4.2150, -69.9406),
    'Medellín': (6.2442, -75.5812),
    'Arauca': (7.0844, -70.7591),
    'Barranquilla': (10.9685, -74.7813),
    'Cartagena de Indias': (10.3910, -75.4794),
    'Tunja': (5.5353, -73.3678),
    'Manizales': (5.0703, -75.5138),
    'Florencia': (1.6144, -75.6062),
    'Yopal': (5.3378, -72.3959),
    'Popayán': (2.4448, -76.6147),
    'Valledupar': (10.4631, -73.2532),
    'Quibdó': (5.6947, -76.6611),
    'Montería': (8.7479, -75.8814),
    'Bogotá': (4.7110, -74.0721),
    'Inírida': (3.8653, -67.9235),
    'San José del Guaviare': (2.5667, -72.6389),
    'Neiva': (2.9273, -75.2819),
    'Riohacha': (11.5444, -72.9073),
    'Santa Marta': (11.2408, -74.1990),
    'Villavicencio': (4.1420, -73.6266),
    'San Juan de Pasto': (1.2136, -77.2811),
    'Cúcuta': (7.8898, -72.4966),
    'Mocoa': (1.1522, -76.6498),
    'Armenia': (4.5339, -75.6811),
    'Pereira': (4.8087, -75.6906),
    'San Andrés': (12.5847, -81.7004),
    'Bucaramanga': (7.1193, -73.1227),
    'Sincelejo': (9.3047, -75.3978),
    'Ibagué': (4.4389, -75.2322),
    'Cali': (3.4516, -76.5319),
    'Mitú': (1.1983, -70.1730),
    'Puerto Carreño': (6.1894, -67.4856)
}

m = folium.Map(location=[4.7110, -74.0721], zoom_start=6)

with open('distancias_ciudades_colombia.csv', mode='w', newline='') as archivo_csv:
    escritor_csv = csv.writer(archivo_csv)
    escritor_csv.writerow(['Origen', 'Destino', 'Distancia (km)'])

    for ciudad, coord in ciudades.items():
        folium.Marker(location=coord, popup=ciudad).add_to(m)

    for origen, coord_origen in ciudades.items():
        for destino, coord_destino in ciudades.items():
            if origen != destino:
                try:
                    ruta = client.directions(
                        coordinates=[(coord_origen[1], coord_origen[0]), (coord_destino[1], coord_destino[0])],
                        profile='driving-car',
                        format='geojson'
                    )

                    distancia_km = ruta['features'][0]['properties']['segments'][0]['distance'] / 1000

                    escritor_csv.writerow([origen, destino, round(distancia_km, 2)])

                    folium.GeoJson(ruta, name=f'{origen} a {destino}').add_to(m)

                except openrouteservice.exceptions.ApiError as e:
                    if isinstance(e.args[0], dict) and "Route could not be found" in e.args[0].get("error", {}).get("message", ""):
                        escritor_csv.writerow([origen, destino, 'NA'])
                    else:
                        print(f"Error desconocido al obtener la ruta de {origen} a {destino}: {e}")

m.save("mapa_rutas_colombia.html")
print("Mapa guardado como 'mapa_rutas_colombia.html'. Ábrelo en tu navegador para ver las rutas.")
print("Archivo CSV guardado como 'distancias_ciudades_colombia.csv'.")


Error desconocido al obtener la ruta de Leticia a Medellín: 404 ({'error': {'code': 2009, 'message': 'Route could not be found - Unable to find a route between points 1 (-69.9406000 -4.2150000) and 2 (-75.5812000 6.2442000).'}, 'info': {'engine': {'build_date': '2024-10-09T09:23:42Z', 'version': '8.2.0'}, 'timestamp': 1731437932640}})
Error desconocido al obtener la ruta de Leticia a Arauca: 404 ({'error': {'code': 2009, 'message': 'Route could not be found - Unable to find a route between points 1 (-69.9406000 -4.2150000) and 2 (-70.7591000 7.0844000).'}, 'info': {'engine': {'build_date': '2024-10-09T09:23:42Z', 'version': '8.2.0'}, 'timestamp': 1731437932778}})
Error desconocido al obtener la ruta de Leticia a Barranquilla: 404 ({'error': {'code': 2009, 'message': 'Route could not be found - Unable to find a route between points 1 (-69.9406000 -4.2150000) and 2 (-74.7813000 10.9685000).'}, 'info': {'engine': {'build_date': '2024-10-09T09:23:42Z', 'version': '8.2.0'}, 'timestamp': 173

/usr/local/lib/python3.10/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 9th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Error desconocido al obtener la ruta de Cartagena de Indias a Inírida: 404 ({'error': {'code': 2009, 'message': 'Route could not be found - Unable to find a route between points 1 (-75.4794000 10.3910000) and 2 (-67.9235000 3.8653000).'}, 'info': {'engine': {'build_date': '2024-10-09T09:23:42Z', 'version': '8.2.0'}, 'timestamp': 1731438209593}})
Error desconocido al obtener la ruta de Cartagena de Indias a San Andrés: 404 ({'error': {'code': 2009, 'message': 'Route could not be found - Unable to find a route between points 1 (-75.4794000 10.3910000) and 2 (-81.7004000 12.5847000).'}, 'info': {'engine': {'build_date': '2024-10-09T09:23:42Z', 'version': '8.2.0'}, 'timestamp': 1731438211565}})
Error desconocido al obtener la ruta de Cartagena de Indias a Mitú: 404 ({'error': {'code': 2009, 'message': 'Route could not be found - Unable to find a route between points 1 (-75.4794000 10.3910000) and 2 (-70.1730000 1.1983000).'}, 'info': {'engine': {'build_date': '2024-10-09T09:23:42Z', 'versi